# Indicadores de Calidad

En el presente notebook se busca extraer, desde repositorios de datos públicos, datasets referentes a **Tipo de Cambio**, **Temperatura** y **Noticias** para estimar un forecast de demanda.

Para este caso, se elige tomar los siguientes indicadores del dataset

- **Calidad de Datos** (Datos Faltantes y Calidad de Duplicados)
- **En el caso de las noticias:** Se añade un grado de polaridad para identificar el grado de opinión (ya sea positiva o negativa)

## Tipo de Cambio

In [33]:
def calcular_metricas(df):

    total_filas = len(df)
    total_duplicados = df.duplicated().sum()
    total_faltantes = df.isnull().sum().sum()

    print(f'Total Filas:',total_filas)
    print(f'Total Duplicados: {total_duplicados}')
    print(f'Total Faltantes: {total_faltantes}\n')
    print(f'% Duplicados: {(total_duplicados/total_filas):.2f}')
    print(f'% Faltantes: {(total_faltantes/total_filas):.2f}')

In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re

In [18]:
URL_TC = 'https://estadisticas.bcrp.gob.pe/estadisticas/series/diarias/resultados/PD04640PD/html'

In [19]:
# Extraemos lo que hay en la página y, como la respuesta es un html,
# Usamos BeautifulSoup para poder leerlo correctamente
respuesta = requests.get(URL_TC)
soup = BeautifulSoup(respuesta.text, 'html.parser')

In [20]:
# Usamos la función find_all para extraer la tabla
filas = soup.find_all('tr')

# Aquí crearé un par de listas para almacenar los valores
fechas = []
valores = []

# Recorro cada fila de la tabla Soup y los valores los guardo en las listas
for fila in filas:
    fecha = fila.find('td', class_='periodo')
    valor = fila.find('td', class_='dato')
    
    if fecha and valor:
        # Por si acaso, quitamos espacios en blanco
        fechas.append(fecha.text.strip())
        valores.append(valor.text.strip())

In [21]:
# Genero un diccionario para poder crear el dataframe
dict_tc = {
    'fecha':fechas,
    'tipo_de_cambio':valores
}

df = pd.DataFrame(dict_tc)

# Ahora, el problema es que las fechas las guarda con un formato 24Oct97, por lo que haré unas transformaciones adicionales
dict_mes = {
    'Ene':'-01-',
    'Feb':'-02-',
    'Mar':'-03-',
    'Abr':'-04-',
    'May':'-05-',
    'Jun':'-06-',
    'Jul':'-07-',
    'Ago':'-08-',
    'Set':'-09-',
    'Oct':'-10-',
    'Nov':'-11-',
    'Dic':'-12-'
}


In [22]:
# Ver Dataset de Tipo de Cambio
df.head()

,fecha,tipo_de_cambio
0,02Ene97,2.614
1,02Ene97,2.614
2,03Ene97,2.620
3,06Ene97,2.628
4,07Ene97,2.635


In [34]:
calcular_metricas(df=df)

Total Filas: 7372
Total Duplicados: 1
Total Faltantes: 0

% Duplicados: 0.00
% Faltantes: 0.00


## Noticias

In [27]:
import requests
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import pandas as pd
from datetime import datetime
from textblob import TextBlob
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch

In [28]:
# Usa un modelo multilingüe para sentimiento
analizador = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    framework="pt"
)

Device set to use cpu


In [29]:
URL_ECONOMIA = 'https://gestion.pe/economia/'
URL_POLITICA = 'https://gestion.pe/peru/politica/'
URLS = [URL_ECONOMIA, URL_POLITICA]

headers = {'User-Agent': 'Mozilla/5.0'}

fecha_actual = datetime.now().strftime('%Y-%m-%d')

In [30]:
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/beto-sentiment-analysis")

def get_polaridad(texto):
    tokens = tokenizer(texto, return_tensors='pt')
    with torch.no_grad():
        output = model(**tokens)
    scores = softmax(output.logits.numpy()[0])
    # Negativo: 0, Neutro: 1, Positivo: 2
    polaridad = scores[2] - scores[0]  # Positivo - Negativo
    return polaridad

In [49]:
import spacy

conceptos = {
    "economía": ["economía", "mercado", "PIB", "crecimiento", "finanzas"],
    "exportaciones": ["exportaciones", "comercio", "aduanas", "envíos", "productos"],
    "Perú": ["Perú", "peruano", "Lima", "andino"],
    "precios internacionales": ["precios", "internacionales", "dólar", "mercado global", "commodities"],
    "inflación": ["inflación", "IPC", "costos", "subida", "alza"]
}

nlp = spacy.load("es_core_news_md")

def extraer_conceptos(texto):
    doc = nlp(texto.lower())
    encontrados = set()
    tokens = [token.text for token in doc]
    for concepto, palabras in conceptos.items():
        if any(p in tokens for p in palabras):
            encontrados.add(concepto)
    return list(encontrados)

c:\Users\aperalesc\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_md' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [53]:
# Lista para almacenar los datos
datos = []

# Procesar titulares de cada URL
for url in URLS:
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraer titulares
    titulares = soup.find_all('h2')
    titulares_texto = [h2.get_text(strip=True) for h2 in titulares]

    for titular in titulares_texto:
        polaridad = get_polaridad(titular)
        conceptos_relacionados = extraer_conceptos(titular)
        datos.append({
            'fecha': fecha_actual,
            'titular': titular,
            'polaridad': polaridad,
            #'conceptos': conceptos_relacionados
        })

# Crear DataFrame
df_titulares = pd.DataFrame(datos)

# Mostrar
print(df_titulares.head())

        fecha                                            titular  polaridad
0  2025-04-04        Mercados e indicadores – viernes 4 de abril   0.017113
1  2025-04-04  Wall Street cae casi 6% en su peor día desde 2...  -0.998647
2  2025-04-04  Celima alista nuevas inversiones en su fábrica...   0.017958
3  2025-04-04  Congreso realizará un pleno económico y produc...  -0.000970
4  2025-04-04  Cambios en Corpac: las tareas pendientes de la...  -0.000985


In [51]:
df_titulares.sort_values(by='polaridad').to_csv(f'Noticias {fecha_actual}.csv',index=False)

In [55]:
calcular_metricas(df=df_titulares)

Total Filas: 110
Total Duplicados: 1
Total Faltantes: 0

% Duplicados: 0.01
% Faltantes: 0.00


In [41]:
# Clave API almacenada en un script de Python
from api_key import API_KEY

# Temperatura

In [42]:
import requests
import pandas as pd
from datetime import datetime, timedelta

ciudades = ('Lima','Trujillo','Cusco','Arequipa','Piura')
DIAS = 7  # últimos 5 días

hoy = datetime.today()
fechas = [(hoy - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, DIAS + 1)]

datos_clima = []

for fecha in fechas:

    for ciudad in ciudades:
        URL = f'http://api.weatherapi.com/v1/history.json?key={API_KEY}&q={ciudad}&dt={fecha}'
        response = requests.get(URL)

        if response.status_code == 200:
            data = response.json()
            dia = data['forecast']['forecastday'][0]['day']
            datos_clima.append({
                'fecha': fecha,
                'ciudad': ciudad,
                'condicion': dia['condition']['text'],
                'temp_max_c': dia['maxtemp_c'],
                'temp_min_c': dia['mintemp_c'],
                'humedad': dia['avghumidity'],
                'lluvia_mm': dia['totalprecip_mm']
            })
        else:
            print(f"Error en la fecha {fecha}: {response.status_code}")

# Convertir a DataFrame
df_temperatura = pd.DataFrame(datos_clima)
print(df_temperatura)

         fecha    ciudad                       condicion  temp_max_c  \
0   2025-04-03      Lima                   Partly cloudy        22.8   
1   2025-04-03  Trujillo            Patchy rain possible        23.0   
2   2025-04-03     Cusco  Patchy light rain with thunder        14.0   
3   2025-04-03  Arequipa            Patchy rain possible        17.4   
4   2025-04-03     Piura                   Partly cloudy        31.5   
5   2025-04-02      Lima                        Overcast        25.2   
6   2025-04-02  Trujillo                   Partly cloudy        23.6   
7   2025-04-02     Cusco   Moderate or heavy rain shower        11.8   
8   2025-04-02  Arequipa            Patchy rain possible        18.0   
9   2025-04-02     Piura            Patchy rain possible        30.5   
10  2025-04-01      Lima                   Partly cloudy        25.2   
11  2025-04-01  Trujillo                          Cloudy        23.0   
12  2025-04-01     Cusco   Moderate or heavy rain shower        

In [43]:
calcular_metricas(df_temperatura)

Total Filas: 35
Total Duplicados: 0
Total Faltantes: 0

% Duplicados: 0.00
% Faltantes: 0.00
